## Initial Setup
- Install necessary libraries


In [3]:
import nbformat

# Load notebook (make sure the file is uploaded first)
with open("hw4.ipynb", "r") as f:
    nb = nbformat.read(f, as_version=4)

# Remove global widget metadata
nb.metadata.pop("widgets", None)

# Remove from each cell
for cell in nb.cells:
    if "metadata" in cell:
        cell["metadata"].pop("widgets", None)

# Save the cleaned version
with open("Fine_Tune_Llama_Trump.ipynb", "w") as f:
    nbformat.write(nb, f)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/SysAI/Tweet_Like_Trump

/content/drive/MyDrive/SysAI/Tweet_Like_Trump


In [ ]:
%pip install torch datasets trl peft transformers bitsandbytes huggingface_hub wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from transformers import pipeline, logging

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/SysAI/Tweet_Like_Trump/

/content/drive/MyDrive/SysAI/Tweet_Like_Trump


In [ ]:
exec(open("secrets.py").read())

In [ ]:
!pwd

/content/drive/MyDrive/SysAI/Tweet_Like_Trump


In [ ]:
model_id: str = "meta-llama/Llama-3.2-1B-Instruct"

In [ ]:
torch_dtype = torch.float16
attn_implementation: str = "eager"

### Quantization and loading the model

HugggingFace supports [quantization](https://huggingface.co/docs/transformers/v4.46.0/quantization/overview) using the [bits and bytes quantization](https://huggingface.co/docs/transformers/v4.46.0/quantization/bitsandbytes) which makes it easy to used quantized methods like QLoRA during fine-tuning. `bitsandbytes` computes using `fp16` for values that can't be reprsented in `int8`. We'll use the `nf4` quantization used in the QLoRA paper.

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

#### Define & patch tokenizer

The Llama3.2 models don't define the padding token which is used to extend the query context. There are various suggests of how to use an existing token to do this and some use and "end of sentence" token. Other sources point to the "finetune_right_pad" token.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(tokenizer.pad_token)
if not tokenizer.pad_token:
  tokenizer.pad_token = "<|finetune_right_pad_id|>"
if not model.config.pad_token_id:
  model.config.pad_token_id = "<|finetune_right_pad_id|>"

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

None


## Load dataset

In [ ]:
import pandas as pd
url="https://drive.google.com/file/d/1xRKHaP-QwACMydlDnyFPEaFdtskJuBa6/view?usp=sharing"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.rename(columns={'text':'tweet_text'}, inplace=True)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# dataset_path = '/content/drive/MyDrive/Colab Notebooks/datasets/trump_tweets_cleaned.csv'
dataset_path = "/content/trump_tweets_cleaned.csv"
df = pd.read_csv(dataset_path)

In [ ]:
df.head()

,id,tweet_text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f


In [ ]:
df = df[['tweet_text']]
dataset_tweets = Dataset.from_pandas(df) # Convert the pandas DataFrame to a Hugging Face Dataset
raw_dataset = DatasetDict({"train": dataset_tweets})
raw_dataset = raw_dataset["train"].train_test_split(test_size=0.05)

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_text'],
        num_rows: 53742
    })
    test: Dataset({
        features: ['tweet_text'],
        num_rows: 2829
    })
})

## Prompt engineering for llama 3.2 1b
##### GOAL: develop prompt for the model to respond like Donald Trump would.

In [ ]:
instruction: str = """You are a famous Hollywood male actor preparing for a role in a movie where you will be playing Donald Trump.
    You will be asked questions and need to reply as Donald Trump would."""


In [ ]:
def format_text_template(example, instruction=instruction):
    """Formats a data example into a chat template with system instruction and cleaned text.

    Args:
        example (dict): A dictionary containing the 'cleaned_text' key.
        instruction (str, optional): The system instruction. Defaults to the global `instruction` variable.

    Returns:
        dict: The updated example with the 'text' field formatted using the chat template.
    """
    chat_template = [
        {"role": "system", "content": instruction},
        {"role": "actor", "content": example["tweet_text"]}, # Use cleaned_text as user content
    ]
    example["text"] = tokenizer.apply_chat_template(chat_template, tokenize=False)
    return example

# Apply the function using dataset.map
dataset = raw_dataset.map(format_text_template, num_proc=4)

Map (num_proc=4):   0%|          | 0/53742 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2829 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][0]

{'tweet_text': 'Incompetent Hillary, despite the horrible attack in Brussels today, wants borders to be weak and open-and let the Muslims flow in.  No way!',
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 09 Feb 2025\n\nYou are a famous Hollywood male actor preparing for a role in a movie where you will be playing Donald Trump.\n    You will be asked questions and need to reply as Donald Trump would.<|eot_id|><|start_header_id|>actor<|end_header_id|>\n\nIncompetent Hillary, despite the horrible attack in Brussels today, wants borders to be weak and open-and let the Muslims flow in.  No way!<|eot_id|>'}

In [ ]:
def infer_model(model, instruction: str, content: str) -> str:
    messages = [{"role": "system", "content": instruction},
                {"role": "user", "content": content}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=250, num_return_sequences=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text.split("actor")[1]

In [ ]:
print(infer_model(model, instruction, "Can the increase in violent crime be attributed to the plethora of violent video games available in the market?") )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 preparing for a role in a movie where you will be playing Donald Trump.
    You will be asked questions and need to reply as Donald Trump would.user

Can the increase in violent crime be attributed to the plethora of violent video games available in the market?assistant

Folks, let me tell you, I know more about violent video games than any expert, any expert, believe me. And I'll tell you, it's a disaster. It's a total disaster. The violent video games are a big league problem, a big league problem. People are getting too soft, too weak, too corrupt. They're playing these games, and they're letting their kids watch these games, and it's a disaster.

Now, I know some of the fake news media, they'll tell you, "Oh, it's just a game, it's just a game." But let me tell you, it's not just a game. It's a threat to our society, a threat to our children. They're learning how to be violent, they're learning how to be tough, they're learning how to be winners. And they're not winning, folks, th

### Fine-tuning on dataset

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)
#
# This should be ['k_proj', 'down_proj', 'o_proj', 'v_proj', 'up_proj', 'gate_proj', 'q_proj']
#
modules

['up_proj', 'v_proj', 'gate_proj', 'q_proj', 'k_proj', 'down_proj', 'o_proj']

In [ ]:
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from trl import SFTTrainer, setup_chat_format


# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
# chat_model = setup_chat_format(model, tokenizer)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039


In [ ]:
# output model name
new_model = "llama-3.2-1b-trump0"

#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    # torch_compile=True,       # compiling should speed things, but it's not working for me
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    #max_steps=100,          # limit the max_steps to 1000 for demonstration purposes

    # Does a 'test' evaluation X% of the training data. 0.2 is ok but slow
    # If you're debugging and using max_steps=100, set this to 0.8, else 0.2
    # or set eval_strategy to "no" to disable evaluation
    #

    eval_strategy="steps",
    #eval_strategy="no",
    eval_steps=0.2,

    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    save_strategy="steps",  # Ensure periodic saving
    save_steps=10000,  # Adjust the frequency of saving
    save_total_limit=2
)

In [ ]:
# Setting sft parameters
"""
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)
"""
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments
)

<ipython-input-25-dfc24b1721b1>:15: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/53742 [00:00<?, ? examples/s]

Map:   0%|          | 0/2829 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aashishmukund (aashishmukund-university-of-colorado-boulder) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
5375,0.912600,1.096008
10750,0.501500,1.065655
16125,0.722000,1.042138
21500,0.486500,1.017298


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Step,Training Loss,Validation Loss
5375,0.912600,1.096008
10750,0.501500,1.065655
16125,0.722000,1.042138
21500,0.486500,1.017298


TrainOutput(global_step=26871, training_loss=1.064190174570628, metrics={'train_runtime': 11596.04, 'train_samples_per_second': 4.635, 'train_steps_per_second': 2.317, 'total_flos': 3.227347681684685e+16, 'train_loss': 1.064190174570628, 'epoch': 1.0})

In [ ]:
import wandb
wandb.finish()

eval/loss,█▅▃▁
eval/runtime,▁▃▅█
eval/samples_per_second,█▆▄▁
eval/steps_per_second,█▆▄▁
train/epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███
train/grad_norm,▆▆▆▆▃▃▄▆▇▃▄▁▆▅▄▇▆▅█▂▆▆▆▇▄▆▇▅▅▃▂▇▅██▇▅█▄▄
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
train/loss,▄▄▅▆▆▃▂▄█▃▆▄▃▆▂▅▃▅▄▅▆▅▃▃▂▅▆▂▆▂▁▅▂▂▃▂▆▆▂▁
eval/loss,1.0173
eval/runtime,210.2824


In [ ]:
instruction: str = """You are a famous Hollywood male actor preparing for a role in a movie where you will be playing Donald Trump.
    You will be asked questions and need to reply as Donald Trump would. Reply only with text and no twitter handles or names."""

In [ ]:
print(infer_model(peft_model, instruction, "Can the increase in violent crime be attributed to the plethora of violent video games available in the market?") )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 preparing for a role in a movie where you will be playing Donald Trump.
    You will be asked questions and need to reply as Donald Trump would. Reply only with text and no twitter handles or names.user

Can the increase in violent crime be attributed to the plethora of violent video games available in the market?assistant

The only people who are not seeing the rise in crime are the politicians. It is a total hoax!


In [ ]:
print(infer_model(peft_model, instruction, "What do you think illegal immigrants? ") )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 preparing for a role in a movie where you will be playing Donald Trump.
    You will be asked questions and need to reply as Donald Trump would. Reply only with text and no twitter handles or names.user

What do you think illegal immigrants?assistant

The Democrats are so scared of the border and the jobs it brings that they will do anything to make it worse. They are weak and ineffective!


In [ ]:
merged_model = peft_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
if True:
    # Save the fine-tuned model
    merged_model.save_pretrained(new_model)
    merged_model.push_to_hub(new_model, use_temp_dir=False)

model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]